<a href="https://colab.research.google.com/github/banshee0716/Financial-Big-Data-Analysis/blob/master/%E9%87%91%E8%9E%8D%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90w13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from typing import List, Tuple
import random

class PortfolioOptimizer:
    def __init__(self,
                 returns: pd.DataFrame,
                 population_size: int = 100,
                 generations: int = 50,
                 mutation_rate: float = 0.01):
        """
        初始化投資組合優化器

        參數:
            returns: 股票報酬率資料
            population_size: 族群大小
            generations: 演算代數
            mutation_rate: 突變率
        """
        self.returns = returns
        self.n_stocks = len(returns.columns)
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def initialize_population(self) -> List[np.ndarray]:
        """
        初始化族群，每個染色體代表一個投資組合
        """
        population = []
        for _ in range(self.population_size):
            # 隨機生成0/1序列，表示是否選擇該股票
            chromosome = np.random.randint(2, size=self.n_stocks)
            # 確保至少選擇一支股票
            if sum(chromosome) == 0:
                chromosome[np.random.randint(self.n_stocks)] = 1
            population.append(chromosome)
        return population

    def calculate_fitness(self, chromosome: np.ndarray) -> Tuple[float, float]:
        """
        計算適應度：根據報酬率和風險評估投資組合
        """
        # 選出的股票報酬率
        selected_returns = self.returns.iloc[:, chromosome == 1]

        if len(selected_returns.columns) == 0:
            return -np.inf, np.inf

        # 假設等權重投資
        weights = np.ones(len(selected_returns.columns)) / len(selected_returns.columns)

        # 計算投資組合報酬率和風險
        portfolio_returns = np.sum(selected_returns.mean() * weights) * 252  # 年化報酬率
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(selected_returns.cov() * 252, weights)))

        # 夏普比率作為適應度（假設無風險利率為0）
        sharpe_ratio = portfolio_returns / portfolio_risk if portfolio_risk != 0 else -np.inf

        return sharpe_ratio, portfolio_risk

    def selection(self, population: List[np.ndarray]) -> List[np.ndarray]:
        """
        使用輪盤法選擇優秀的染色體
        """
        fitness_scores = [self.calculate_fitness(chrom)[0] for chrom in population]
        fitness_scores = np.array(fitness_scores)

        # 將適應度轉換為正數以進行輪盤選擇
        min_fitness = min(fitness_scores)
        if min_fitness < 0:
            fitness_scores = fitness_scores - min_fitness + 1e-6

        # 計算選擇機率
        probs = fitness_scores / sum(fitness_scores)

        # 選擇新的族群
        new_population = []
        for _ in range(self.population_size):
            selected = np.random.choice(len(population), p=probs)
            new_population.append(population[selected].copy())

        return new_population

    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        執行單點交配
        """
        point = np.random.randint(1, self.n_stocks)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])

        # 確保至少選擇一支股票
        if sum(child1) == 0:
            child1[np.random.randint(self.n_stocks)] = 1
        if sum(child2) == 0:
            child2[np.random.randint(self.n_stocks)] = 1

        return child1, child2

    def mutation(self, chromosome: np.ndarray) -> np.ndarray:
        """
        執行突變
        """
        for i in range(len(chromosome)):
            if np.random.random() < self.mutation_rate:
                chromosome[i] = 1 - chromosome[i]

        # 確保至少選擇一支股票
        if sum(chromosome) == 0:
            chromosome[np.random.randint(self.n_stocks)] = 1

        return chromosome

    def optimize(self) -> Tuple[np.ndarray, float, float]:
        """
        執行基因演算法優化流程
        """
        # 初始化族群
        population = self.initialize_population()

        # 迭代進化
        for generation in range(self.generations):
            # 選擇
            population = self.selection(population)

            # 交配
            new_population = []
            for i in range(0, self.population_size, 2):
                if i + 1 < self.population_size:
                    child1, child2 = self.crossover(population[i], population[i+1])
                    new_population.extend([child1, child2])
                else:
                    new_population.append(population[i])

            # 突變
            population = [self.mutation(chrom) for chrom in new_population]

            # 追蹤最佳解
            best_chromosome = max(population, key=lambda x: self.calculate_fitness(x)[0])
            best_fitness, best_risk = self.calculate_fitness(best_chromosome)

            print(f"Generation {generation + 1}: Best Sharpe Ratio = {best_fitness:.4f}")

        # 找出最終最佳解
        best_chromosome = max(population, key=lambda x: self.calculate_fitness(x)[0])
        best_fitness, best_risk = self.calculate_fitness(best_chromosome)

        return best_chromosome, best_fitness, best_risk


In [5]:
# 產生示範用的股票報酬率數據
np.random.seed(42)
n_stocks = 20
n_days = 252

# 模擬每日報酬率數據
returns_data = np.random.normal(0.001, 0.02, (n_days, n_stocks))
stock_names = [f"Stock_{i+1}" for i in range(n_stocks)]
returns_df = pd.DataFrame(returns_data, columns=stock_names)

# 建立並執行優化器
optimizer = PortfolioOptimizer(returns_df)
best_portfolio, sharpe_ratio, risk = optimizer.optimize()

# 輸出結果
selected_stocks = [stock_names[i] for i in range(n_stocks) if best_portfolio[i] == 1]
print("\nOptimization Results:")
print(f"Selected Stocks: {selected_stocks}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Portfolio Risk: {risk:.4f}")

Generation 1: Best Sharpe Ratio = 4.3366
Generation 2: Best Sharpe Ratio = 4.8782
Generation 3: Best Sharpe Ratio = 4.3631
Generation 4: Best Sharpe Ratio = 4.7154
Generation 5: Best Sharpe Ratio = 4.4696
Generation 6: Best Sharpe Ratio = 4.8291
Generation 7: Best Sharpe Ratio = 4.7923
Generation 8: Best Sharpe Ratio = 4.6916
Generation 9: Best Sharpe Ratio = 5.0426
Generation 10: Best Sharpe Ratio = 5.0426
Generation 11: Best Sharpe Ratio = 5.0029
Generation 12: Best Sharpe Ratio = 4.6318
Generation 13: Best Sharpe Ratio = 4.6834
Generation 14: Best Sharpe Ratio = 4.8349
Generation 15: Best Sharpe Ratio = 4.8192
Generation 16: Best Sharpe Ratio = 4.8192
Generation 17: Best Sharpe Ratio = 4.8964
Generation 18: Best Sharpe Ratio = 4.8192
Generation 19: Best Sharpe Ratio = 5.1538
Generation 20: Best Sharpe Ratio = 5.1538
Generation 21: Best Sharpe Ratio = 5.3157
Generation 22: Best Sharpe Ratio = 4.9862
Generation 23: Best Sharpe Ratio = 4.9862
Generation 24: Best Sharpe Ratio = 5.2432
G

In [1]:
import numpy as np
import random
from scipy.optimize import minimize

# 模擬資產的回報率和協方差矩陣
np.random.seed(42)
num_assets = 4  # 假設投資組合中有 4 種資產
mean_returns = np.random.uniform(0.01, 0.1, num_assets)  # 資產平均回報率
cov_matrix = np.random.uniform(0.0005, 0.0015, (num_assets, num_assets))
cov_matrix = (cov_matrix + cov_matrix.T) / 2  # 確保協方差矩陣對稱
np.fill_diagonal(cov_matrix, np.random.uniform(0.001, 0.002, num_assets))
risk_free_rate = 0.02  # 無風險利率

# 計算投資組合的夏普比率
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate):
    returns = np.sum(mean_returns * weights)
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (returns - risk_free_rate) / std_dev
    return returns, std_dev, sharpe_ratio

# 基因演算法設計
def genetic_algorithm(num_generations, population_size, num_assets, mean_returns, cov_matrix, risk_free_rate):
    # 隨機生成初始族群
    def create_population(size, num_assets):
        return [np.random.dirichlet(np.ones(num_assets)) for _ in range(size)]

    # 適應度函數（使用夏普比率作為目標）
    def fitness(weights):
        _, _, sharpe_ratio = portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)
        return sharpe_ratio

    # 交配（生成新一代）
    def crossover(parent1, parent2):
        child = np.array([random.choice(gene_pair) for gene_pair in zip(parent1, parent2)])
        return child / child.sum()

    # 突變
    def mutate(weights):
        idx = random.randint(0, len(weights) - 1)
        weights[idx] = np.random.uniform(0, 1)
        return weights / weights.sum()

    # 初始化族群
    population = create_population(population_size, num_assets)

    for generation in range(num_generations):
        # 評估適應度
        fitness_scores = [(individual, fitness(individual)) for individual in population]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)

        # 選擇適者
        top_individuals = fitness_scores[:population_size // 2]

        # 生成下一代
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample([ind[0] for ind in top_individuals], 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.1:  # 10% 的突變機率
                child = mutate(child)
            new_population.append(child)
        population = new_population

    # 返回最佳個體
    best_individual = max(population, key=fitness)
    best_returns, best_std_dev, best_sharpe_ratio = portfolio_performance(best_individual, mean_returns, cov_matrix, risk_free_rate)
    return best_individual, best_returns, best_std_dev, best_sharpe_ratio


In [2]:
# 使用基因演算法找出最佳投資組合權重
best_weights, best_returns, best_std_dev, best_sharpe_ratio = genetic_algorithm(
    num_generations=100,
    population_size=50,
    num_assets=num_assets,
    mean_returns=mean_returns,
    cov_matrix=cov_matrix,
    risk_free_rate=risk_free_rate
)

# 輸出結果
print("最佳權重:", best_weights)
print("投資組合年化回報率:", best_returns)
print("投資組合年化波動率:", best_std_dev)
print("投資組合夏普比率:", best_sharpe_ratio)


最佳權重: [0.00097907 0.7853714  0.21046405 0.00318548]
投資組合年化回報率: 0.09126963554492054
投資組合年化波動率: 0.031192383489649273
投資組合夏普比率: 2.284840963454117
